# Primitivas Geométricas y Transformaciones

In [ ]:
import numpy as np
import cv2

In [7]:
def muestra_imagen(imagen):
    cv2.imshow("Canvas base", imagen)
    cv2.waitKey(0)

In [61]:
w = 250
h = 250
imagen_base = np.zeros((w, h))
muestra_imagen(imagen_base)

In [22]:
def obtener_homogeneas(coordenadas_2D, w):
    return np.hstack([coordenadas_2D, w])

def obtener_2D(coordenadas_homogeneas):
    return cv2.convertPointsFromHomogeneous(coordenadas_homogeneas)

In [62]:
# Slicing
imagen_base[int(w/2-25):int(w/2+25), int(h/2-25):int(h/2+25)] = 100
muestra_imagen(imagen_base)

x = 100
y = 100
escala = 1

# Punto 2D
p = [x, y]
# Punto 2D Coordenadas Homogeneas
p_h = np.array([[x, y, escala]])

print(obtener_homogeneas(p, escala))
print(obtener_2D(p_h))

[100 100   1]
[[[100. 100.]]]


# Transformaciones Euclideanas

In [168]:
class TransformacionesEuclideanas:
    @staticmethod
    def rotacion(angulo):
        return np.array([
            [np.cos(angulo), -np.sin(angulo), 0],
            [np.sin(angulo), np.cos(angulo), 1],
            [0, 0, 1]
        ])
    
    @staticmethod
    def traslado(x,y):
        return np.array([
            [1, 0, x],
            [0, 1, y],
            [0, 0, 1]
        ])
    
    @staticmethod
    def reflexion(eje='x'):
        if eje == 'x':  # Refleja sobre eje Y
            return np.array([[-1, 0, 0],
                             [0, 1, 0],
                             [0, 0, 1]])
        elif eje == 'y':  # Refleja sobre eje X
            return np.array([[1, 0, 0],
                             [0, -1, 0],
                             [0, 0, 1]])
        elif eje == 'origen':  # Refleja sobre origen
            return np.array([[-1, 0, 0],
                             [0, -1, 0],
                             [0, 0, 1]])

In [ ]:
imagen_transformada = np.zeros((w, h), dtype=imagen_base.dtype)

R = TransformacionesEuclideanas.rotacion(np.pi/16)
T1 = TransformacionesEuclideanas.traslado(-125, -125)
T2 = TransformacionesEuclideanas.traslado(125, 125)

#T = traslado2 @ rotacion @ traslado
T = T2 @ R @ T1

M_inv = np.linalg.inv(T)
for x in range(w):
    for y in range(h):
        p_destino_h = np.array([x, y, 1])
        p_origen_h = M_inv @ p_destino_h

        x_origen = p_origen_h[0] / p_origen_h[2]
        y_origen = p_origen_h[1] / p_origen_h[2]

        #Si está dentro de la imagen original
        if 0 <= x_origen < w and 0 <= y_origen < h:
        # Interpolación por vecino más cercano
            x_idx = int(round(x_origen))%h
            y_idx = int(round(y_origen))%w
            imagen_transformada[y, x] = imagen_base[y_idx, x_idx]
            
muestra_imagen(imagen_transformada)